In [2]:
import pandas as pd
import numpy as np
from neuralprophet import NeuralProphet
import matplotlib.pyplot as plt

C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.


In [5]:
df_tb_productos = pd.read_csv('./data/tb_productos.txt', sep='\t')
df_sell_in = pd.read_csv('./data/sell-in.txt', sep='\t')
df_predecir = pd.read_csv('./data/productos_a_predecir.txt', sep='\t')
df_tb_stocks = pd.read_csv('./data/tb_stocks.txt', sep='\t')

In [6]:
df_sell_in[df_sell_in['periodo']>=201712]

,periodo,customer_id,product_id,plan_precios_cuidados,cust_request_qty,cust_request_tn,tn
970166,201712,10037,20524,0,2,0.04543,0.04543
970167,201712,10117,20524,0,1,0.04543,0.04543
970168,201712,10067,20524,0,3,0.18171,0.18171
970169,201712,10329,20524,0,1,0.03028,0.03028
970170,201712,10035,20524,0,1,0.02271,0.02271
...,...,...,...,...,...,...,...
2945813,201912,10105,20853,0,1,0.02230,0.02230
2945814,201912,10092,20853,0,1,0.00669,0.00669
2945815,201912,10006,20853,0,7,0.02898,0.02898
2945816,201912,10018,20853,0,4,0.01561,0.01561


In [7]:
# Join tb_productos to sell_in on product_id
########3df_sell_in_merged = pd.merge(df_sell_in, df_tb_productos, on='product_id', how='left')
# Join tb_stocks to sell_in_merged on both product_id and periodo
########df_final = pd.merge(df_sell_in_merged, df_tb_stocks, on=['product_id', 'periodo'], how='left')

In [8]:
productos_a_predecir = df_predecir['product_id'].unique()

df_final = df_sell_in[df_sell_in['product_id'].isin(productos_a_predecir)]

In [9]:
df_final['fecha'] = pd.to_datetime(df_final['periodo'], format='%Y%m')

WARNING - (py.warnings._showwarnmsg) - C:\Users\Admin\AppData\Local\Temp\ipykernel_20264\1590532408.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['fecha'] = pd.to_datetime(df_final['periodo'], format='%Y%m')



In [10]:
df_final = df_final.groupby(['product_id','fecha'])['tn'].sum().reset_index()

In [11]:
#df_final.set_index('fecha', inplace=True)
#df_final.index = df_final.index#.to_period('M')

In [12]:
df_final_filtrado = df_final[df_final['product_id'].isin(productos_a_predecir)]
df_final_filtrado['fecha'] = pd.to_datetime(df_final_filtrado['fecha'])

In [13]:
forecast_results = {}
predictions = []
# Ajustar y predecir para cada producto
for producto in df_final_filtrado['product_id'].unique():
    # Filtrar los datos del producto
    df_producto = df_final_filtrado[df_final_filtrado['product_id'] == producto]
    print(producto)
    df_producto = df_producto[['fecha', 'tn']]  # Asegúrate de que 'date' es la columna de fechas y 'tn' es el valor a predecir
    
    # Renombrar columnas para NeuralProphet
    df_producto = df_producto.rename(columns={'fecha': 'ds', 'tn': 'y'})
    
    # Convertir la columna 'ds' a datetime si no lo está
    df_producto['ds'] = pd.to_datetime(df_producto['ds'])

    try:
        # Crear y ajustar el modelo NeuralProphet
        model = NeuralProphet()
        model.fit(df_producto, freq='M')  # Ajusta la frecuencia a 'M' para mensual
    
        # Realizar el pronóstico para 2 meses adelante
        forecast_steps = 2
        future = model.make_future_dataframe(df_producto, periods=forecast_steps)
        forecast = model.predict(future)
        
        # Obtener la predicción del segundo mes
        second_month_prediction = forecast.iloc[1]['yhat1']  # .iloc[1] obtiene el valor predicho para el segundo mes
    
        # Almacenar el product_id y la predicción en la lista
        predictions.append({'product_id': producto, 'tn': second_month_prediction})
    except Exception as e:
        print(f"Error al procesar el producto {producto}: {e}")
        predictions.append({'product_id': producto, 'tn': df_producto['y'].mean()})
        continue

# Convertir la lista a un DataFrame
df_predictions = pd.DataFrame(predictions)

WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)



INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.222]% of the data.
WARNING - (py.warnings._showwarnmsg) - C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\neuralprophet\df_utils.py:1173: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  aux_ts = pd.DataFrame(pd.date_range("1994-01-01", periods=100, freq=freq_str))

WARNING - (py.warnings._showwarnmsg) - C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFou

20001


Finding best initial lr: 100%|██████████| 203/203 [00:00<00:00, 293.35it/s]


Epoch 380: 100%|██████████| 380/380 [00:00<00:00, 15971.74it/s, loss=0.0349, v_num=0, MAE=150.0, RMSE=175.0, Loss=0.0368, RegLoss=0.000] 

...

INFO - (NP.df_utils.return_df_in_original_format) - Returning df with no ID column


WARNING - (NP.forecaster.fit) - When Global modeling with local normalization, metrics are displayed in normalized scale.
WARNING - (py.warnings._showwarnmsg) - C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Major frequency MS corresponds to [97.222]% of the data.
WARNING - (py.warnings._showwarnmsg) - C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\neuralprophet\df_utils.py:1173: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  aux_ts = pd.DataFrame(pd.date_range("1994-01-01", pe

20362


WARNING - (py.warnings._showwarnmsg) - C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

WARNING - (py.warnings._showwarnmsg) - C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\neuralprophet\df_utils.py:1152: FutureWarning: Series.view is deprecated and will be removed in a future version. Use ``astype`` as an alternative to change the dtype.
  converted_ds = pd.to_datetime(ds_col, utc=True).view(dtype=np.int64)

INFO - (NP.df_utils._infer_frequency) - Defined frequency is equal to major frequency - MS
INFO - (NP.config.init_data_params) - Setting normalization to global as 

Epoch 56:  15%|█▍        | 56/380 [00:00<00:00, 51083.30it/s, loss=0.0369, v_num=297, MAE=3.200, RMSE=4.020, Loss=0.0418, RegLoss=0.000]

In [ ]:
# Convertir la lista a un DataFrame
df_predictions = pd.DataFrame(predictions)
df_predictions = df_predictions.fillna(0)
df_predictions.to_csv('predicciones_NEURAL_PROPHET.csv', index=False, header=True)